In [0]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re

In [0]:
# from google.colab import drive
# drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
# import os
# os.chdir('/content/drive/My Drive/')
!git clone https://github.com/jessicaw9910/ANGSD.git

Cloning into 'ANGSD'...
remote: Enumerating objects: 55, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 55 (delta 10), reused 47 (delta 6), pack-reused 0
Unpacking objects: 100% (55/55), done.


# GSM Number and Links

In [27]:
result = requests.get("https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE40419")
print(result.status_code)  
print(result.headers)
src = result.content
print(src)
soup = BeautifulSoup(src, 'lxml')
links = soup.find_all("a")

200
{'Date': 'Thu, 02 Apr 2020 14:14:45 GMT', 'Server': 'Apache', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains; preload', 'Referrer-Policy': 'origin-when-cross-origin', 'Content-Security-Policy': 'upgrade-insecure-requests', 'Cache-Control': 'no-cache', 'NCBI-PHID': '0C42D855E85F35510000000000000001.m_1', 'NCBI-SID': '0C42D855E85F3551_0000SID', 'Pragma': 'no-cache', 'Access-Control-Allow-Methods': 'POST, GET, PUT, OPTIONS, PATCH, DELETE', 'Access-Control-Allow-Origin': '', 'Access-Control-Allow-Credentials': 'true', 'Access-Control-Allow-Headers': 'Origin,X-Accept-Charset,X-Accept,Content-Type,X-Requested-With,NCBI-SID,NCBI-PHID', 'Content-Type': 'text/html', 'Set-Cookie': 'ncbi_sid=0C42D855E85F3551_0000SID; domain=.nih.gov; path=/; expires=Fri, 02 Apr 2021 14:14:45 GMT', 'Vary': 'Accept-Encoding', 'Content-Encoding': 'gzip', 'X-UA-Compatible': 'IE=Edge', 'X-XSS-Protection': '1; mode=block', 'Keep-Alive': 'timeout=1, max=10', 'Connection': 'Keep-Alive', 'Transfer-E

In [0]:
link_gsm_names = []
link_gsm_links = []

for link in links:
  if "GSM" in link.text:
    link_gsm_names.append(link.text)
    link_gsm_links.append(link.attrs['href'])

In [29]:
print(link_gsm_links)
print(link_gsm_names)

['/geo/query/acc.cgi?acc=GSM993606', '/geo/query/acc.cgi?acc=GSM993607', '/geo/query/acc.cgi?acc=GSM993608', '/geo/query/acc.cgi?acc=GSM993610', '/geo/query/acc.cgi?acc=GSM993611', '/geo/query/acc.cgi?acc=GSM993612', '/geo/query/acc.cgi?acc=GSM993613', '/geo/query/acc.cgi?acc=GSM993614', '/geo/query/acc.cgi?acc=GSM993616', '/geo/query/acc.cgi?acc=GSM993617', '/geo/query/acc.cgi?acc=GSM993618', '/geo/query/acc.cgi?acc=GSM993619', '/geo/query/acc.cgi?acc=GSM993620', '/geo/query/acc.cgi?acc=GSM993621', '/geo/query/acc.cgi?acc=GSM993622', '/geo/query/acc.cgi?acc=GSM993623', '/geo/query/acc.cgi?acc=GSM993624', '/geo/query/acc.cgi?acc=GSM993625', '/geo/query/acc.cgi?acc=GSM993626', '/geo/query/acc.cgi?acc=GSM993627', '/geo/query/acc.cgi?acc=GSM993628', '/geo/query/acc.cgi?acc=GSM993629', '/geo/query/acc.cgi?acc=GSM993630', '/geo/query/acc.cgi?acc=GSM993631', '/geo/query/acc.cgi?acc=GSM993632', '/geo/query/acc.cgi?acc=GSM993633', '/geo/query/acc.cgi?acc=GSM993634', '/geo/query/acc.cgi?acc=GSM

In [0]:
samples_df = pd.DataFrame(data=link_gsm_names, columns=["gsm"])

for n in range(len(link_gsm_links)):
  link_gsm_links[n] = "https://www.ncbi.nlm.nih.gov/"+link_gsm_links[n]

samples_df["gsm_links"] = link_gsm_links

samples_df['age_at_diagnosis'] = np.NaN
samples_df['gender'] = np.NaN
samples_df['smoking_status'] = np.NaN
samples_df['stage'] = np.NaN

samples_df.head()

# GEO Characteristics

In [0]:
len(samples_df)

164

In [33]:
for n in range(len(samples_df)):
  # print(n)
  result_gsm = requests.get(samples_df["gsm_links"][n])

  src_gsm = result_gsm.content

  soup_gsm = BeautifulSoup(src_gsm, 'lxml')
  # print(soup_gsm.prettify())
  tables_gsm = soup_gsm.findChildren('table')
  
  # find the last row where gender occurs
  # rows = tables_gsm[0].findChildren(['td'])
  # i = 0
  # for row in rows:
  #  if "gender" in str(row):
  #    n = i
  #  i += 1

  string = str(tables_gsm[0])

  age_at_diagnosis = int(re.search('age_at_diagnosis: (.+?)<br/>', string).group(1))
  gender = re.search('gender: (.+?)<br/>', string).group(1)
  smoking_status = re.search('smoking_status: (.+?)<br/>', string).group(1)
  stage = re.search('Stage: (.+?)<br/>', string).group(1)

  c1 = list(samples_df.columns).index('age_at_diagnosis')
  c2 = list(samples_df.columns).index('gender')
  c3 = list(samples_df.columns).index('smoking_status')
  c4 = list(samples_df.columns).index('stage')

  samples_df.iloc[n, c1] = age_at_diagnosis
  samples_df.iloc[n, c2] = gender
  samples_df.iloc[n, c3] = smoking_status
  samples_df.iloc[n, c4] = stage

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163


In [40]:
samples_df[0:35]['smoking_status'].value_counts()

never smoker      13
smoker            11
current smoker     7
NA                 4
Name: smoking_status, dtype: int64

# GSM Link to ERX

In [0]:
link_erx = []
gsm_erx = []

for m in samples_df['gsm_links']:
  result_gsm = requests.get(m)
  src_gsm = result_gsm.content
  soup_gsm = BeautifulSoup(src_gsm, 'lxml')
  links_gsm = soup_gsm.find_all("a")
  for n in links_gsm:
    if "ERX" in n.text:
      link_erx.append(n.text)
      gsm_erx.append(m)

In [43]:
len(link_erx)

167

In [44]:
len(gsm_erx)
gsm_erx = [x.replace('https://www.ncbi.nlm.nih.gov//geo/query/acc.cgi?acc=', '') for x in gsm_erx]
len(set(gsm_erx))

164

In [45]:
from collections import Counter

temp1 = Counter(gsm_erx).keys() # equals to list(set(list))
temp2 = Counter(gsm_erx).values() # counts the elements' frequency

gsm_erx_dict = dict(zip(temp1, temp2))
duplicates = []

for name, count in gsm_erx_dict.items():
  if count == 2:
    duplicates.append(name)

for n in duplicates:
  matches = []
  i = 0
  for m in gsm_erx:
    if m == n:
      matches.append(i)
    i += 1
  print(matches)

[41, 42]
[51, 52]
[53, 54]


In [46]:
print(link_erx[41])
print(link_erx[42])
print(link_erx[51])
print(link_erx[52])
print(link_erx[53])
print(link_erx[54])
print(duplicates)

ERX140463
ERX140464
ERX140473
ERX140474
ERX140475
ERX140476
['GSM993649', 'GSM993658', 'GSM993659']


In [0]:
experiment_df = pd.DataFrame(data=link_erx, columns=["experiment_accession"])
experiment_df["gsm"] = gsm_erx

In [59]:
experiment_df.head()

,experiment_accession,gsm
0,ERX140427,GSM993606
1,ERX140428,GSM993607
2,ERX140429,GSM993608
3,ERX140430,GSM993610
4,ERX140431,GSM993611


In [60]:
data_1 = pd.read_csv("/content/ANGSD/files/Backup_v01.csv", header = None)
data_1.columns = ['gsm', 'sample']
data_1.head()

,gsm,sample
0,GSM993606,LC_C1
1,GSM993607,LC_C2
2,GSM993608,LC_C3
3,GSM993610,LC_C4
4,GSM993611,LC_C5


In [61]:
data_2 = pd.read_csv("/content/ANGSD/files/experiment_list.txt", sep="\t")
data_2.head()

,study_accession,sample_accession,secondary_sample_accession,experiment_accession,run_accession,fastq_ftp
0,PRJEB2784,SAMEA1094642,ERS077419,ERX036541,ERR058695,ftp.sra.ebi.ac.uk/vol1/fastq/ERR058/ERR058695/...
1,PRJEB2784,SAMEA1094643,ERS079846,ERX040276,ERR062334,NaN
2,PRJEB2784,SAMEA1094641,ERS079847,ERX040277,ERR062335,NaN
3,PRJEB2784,SAMEA1094645,ERS079848,ERX040278,ERR062336,NaN
4,PRJEB2784,SAMEA1094646,ERS079849,ERX040279,ERR062337,NaN


In [62]:
data_2.run_accession.value_counts()

ERR164618    1
ERR164477    1
ERR164529    1
ERR164601    1
ERR164595    1
            ..
ERR164606    1
ERR164580    1
ERR164506    1
ERR164600    1
ERR164554    1
Name: run_accession, Length: 173, dtype: int64

In [0]:
experiment_df = experiment_df.merge(samples_df, on='gsm')
experiment_df = experiment_df.merge(data_1, on='gsm')
experiment_df = experiment_df.merge(data_2, on='experiment_accession')

In [64]:
experiment_df

,experiment_accession,gsm,gsm_links,age_at_diagnosis,gender,smoking_status,stage,sample,study_accession,sample_accession,secondary_sample_accession,run_accession,fastq_ftp
0,ERX140427,GSM993606,https://www.ncbi.nlm.nih.gov//geo/query/acc.cg...,54.0,male,smoker,1A,LC_C1,PRJEB2784,SAMEA1499272,ERS167310,ERR164550,ftp.sra.ebi.ac.uk/vol1/fastq/ERR164/ERR164550/...
1,ERX140428,GSM993607,https://www.ncbi.nlm.nih.gov//geo/query/acc.cg...,51.0,male,current smoker,2B,LC_C2,PRJEB2784,SAMEA1499302,ERS167311,ERR164551,ftp.sra.ebi.ac.uk/vol1/fastq/ERR164/ERR164551/...
2,ERX140429,GSM993608,https://www.ncbi.nlm.nih.gov//geo/query/acc.cg...,65.0,female,never smoker,1A,LC_C3,PRJEB2784,SAMEA1499306,ERS167312,ERR164552,ftp.sra.ebi.ac.uk/vol1/fastq/ERR164/ERR164552/...
3,ERX140430,GSM993610,https://www.ncbi.nlm.nih.gov//geo/query/acc.cg...,52.0,male,never smoker,3B,LC_C4,PRJEB2784,SAMEA1499329,ERS167313,ERR164553,ftp.sra.ebi.ac.uk/vol1/fastq/ERR164/ERR164553/...
4,ERX140431,GSM993611,https://www.ncbi.nlm.nih.gov//geo/query/acc.cg...,68.0,male,NA,3A,LC_C5,PRJEB2784,SAMEA1499346,ERS167314,ERR164554,ftp.sra.ebi.ac.uk/vol1/fastq/ERR164/ERR164554/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,ERX140422,GSM993767,https://www.ncbi.nlm.nih.gov//geo/query/acc.cg...,48.0,female,never smoker,NA,LC_S48_nor,PRJEB2784,SAMEA1499227,ERS158801,ERR164545,ftp.sra.ebi.ac.uk/vol1/fastq/ERR164/ERR164545/...
163,ERX140423,GSM993768,https://www.ncbi.nlm.nih.gov//geo/query/acc.cg...,66.0,female,never smoker,NA,LC_S49_nor,PRJEB2784,SAMEA1499232,ERS158802,ERR164546,ftp.sra.ebi.ac.uk/vol1/fastq/ERR164/ERR164546/...
164,ERX140424,GSM993769,https://www.ncbi.nlm.nih.gov//geo/query/acc.cg...,64.0,female,never smoker,NA,LC_S50_nor,PRJEB2784,SAMEA1499373,ERS158803,ERR164547,ftp.sra.ebi.ac.uk/vol1/fastq/ERR164/ERR164547/...
165,ERX140425,GSM993770,https://www.ncbi.nlm.nih.gov//geo/query/acc.cg...,64.0,male,current smoker,NA,LC_S51_nor,PRJEB2784,SAMEA1499352,ERS158804,ERR164548,ftp.sra.ebi.ac.uk/vol1/fastq/ERR164/ERR164548/...


In [0]:
experiment_df.to_csv(r'/content/ANGSD/files/Backup_v02.csv', index=False)